# AFID protocol analysis

An interactive notebook for processing the results from the tutorial.

In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)

In [2]:
# initialize paths and variables
setwd('~/GitHub/BHG18_afidprotocol//input/input_fid')

df_raters <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        template=factor(),mri_type=factor(),session=integer(),date=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)

csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
  curr_split <- unlist(strsplit(csv_files[i],"_"))
  if (length(curr_split)>1) { # extract name and session data
    rater_template <- curr_split[1]
    rater_mri_type <- curr_split[2]
    rater_name <- curr_split[3]
    rater_session <- as.numeric(curr_split[4])
    rater_date <- as.numeric(unlist(strsplit(curr_split[5],"[.]"))[1])
  }

  curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
  df_rater <- data.frame(fid = 1:length(curr_rater$V1))

  df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                    template=rater_template,mri_type=rater_mri_type,
                    session=rater_session,date=rater_date,name=curr_rater[12],
                    description=curr_rater[13])
  
  df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
  df_raters <- rbind(df_raters,df_rater)
}

# Remapping levels to integers
levels(df_raters$rater) <- 1:length(levels(df_raters$rater))

In [14]:
# load in the gold-standard study mean

# create study specific mean
#ddply(df_raters, "fid", summarize, X=mean(X), Y=mean(Y), Z=mean(Z) ) # study mean

#### Gold-standard study MEAN ########################################################################
setwd('~/GitHub/BHG18_afidprotocol/input/input_mean/')
UHF_gold <- read.table('UHF_MEAN_no_outliers.fcsv', header=FALSE, sep=",")
df_gold <- data.frame(fid = 1:length(curr_rater$V1))
df_gold <- cbind(df_gold,X=UHF_gold[2],Y=UHF_gold[3],Z=UHF_gold[4],rater='GOLD',
                  template=rater_template,mri_type=rater_mri_type,
                  session=rater_session,date=rater_date,name=UHF_gold[12],
                  description=UHF_gold[13])
df_gold <- rename(df_gold, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))

#### Recalculate rater errors ########################################################################

df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA

df_raters$rater_error <- NA
df_raters$outlier <- NA

# TODO: refactor into distance function
for (i in 1:dim(df_raters)[1]) {
        curr_rater <- df_raters[i,]
        mean_raters <- df_gold[curr_rater$fid,]

        xdist <- curr_rater$X - mean_raters$X
        ydist <- curr_rater$Y - mean_raters$Y
        zdist <- curr_rater$Z - mean_raters$Z
        
        rater_error <- sqrt( (curr_rater$X - mean_raters$X)^2 + (curr_rater$Y - mean_raters$Y)^2 + (curr_rater$Z - mean_raters$Z)^2 )
        
        df_raters[i,]$xdist <- xdist
        df_raters[i,]$ydist <- ydist
        df_raters[i,]$zdist <- zdist
        
        rater_error <- sqrt( (curr_rater$X - mean_raters$X)^2 + (curr_rater$Y - mean_raters$Y)^2 + (curr_rater$Z - mean_raters$Z)^2 )
        df_raters[i,]$rater_error <- rater_error
        df_raters[i,]$outlier <- (rater_error > 10) # focus on true outliers (1cm+) first (TODO: optimize threshold later)
}

## Summary Statistics

Summarizing overall mean AFLE from the tutorial and individual rater results.

In [15]:
# summary statistics
sprintf("Overall mean rater error (AFLE): %.2f +/- %.2f mm", mean(df_raters$rater_error), sd(df_raters$rater_error))
df_rater_summary <- ddply(df_raters, "rater", summarize, mean=mean(rater_error), sd=sd(rater_error))
sprintf("The top rater was Rater %d!", which.min(df_rater_summary$mean))
df_rater_summary

[1] "Overall mean rater error (AFLE): 1.62 +/- 2.40 mm"

[1] "The top rater was Rater 3!"

rater,mean,sd
1,2.655555,4.0570655
2,1.465626,2.0840100
3,1.102250,0.6947745
4,1.263015,1.6454577
5,1.187308,0.6604209
6,1.409958,1.6511633
7,1.146385,1.5384989
8,3.353794,4.7147488
9,1.156302,0.7935470
10,1.276090,2.1096329


### Summary across the different AFID points

In [22]:
cbind( ddply(df_raters, .(fid), summarize, mean=mean(rater_error), sd=sd(rater_error)), description = df_gold$description)

fid,mean,sd,description
1,0.3877299,0.1755934,AC
2,0.5119801,0.1948584,PC
3,1.1772151,0.3643899,infracollicular sulcus
4,0.8934505,0.4448808,PMJ
5,1.1816383,0.5767583,superior interpeduncular fossa
6,0.5061952,0.2232138,R superior LMS
7,0.9006822,0.3415488,L superior LMS
8,1.2580568,0.7162415,R inferior LMS
9,1.4334641,0.6892487,L inferior LMS
10,1.0098439,0.3826892,culmen


## Individual Rater Performance

Summarizing results for an individual rater.

In [6]:
i <- 3
df_curr_rater <- subset(df_raters, rater==i)[,c("fid","name","description","rater_error","outlier")]
df_curr_rater$description <- df_gold$description
sprintf("Overall mean rater error (AFLE): %.2f +/- %.2f mm", mean(df_curr_rater$rater_error), sd(df_curr_rater$rater_error))
df_curr_rater

[1] "Overall mean rater error (AFLE): 1.10 +/- 0.69 mm"

,fid,name,description,rater_error,outlier
65,1,AC,AC,0.1355433,FALSE
66,2,PC,PC,0.6884137,FALSE
67,3,3,infracollicular sulcus,1.5301650,FALSE
68,4,4,PMJ,0.6658026,FALSE
69,5,5,superior interpeduncular fossa,1.3824154,FALSE
70,6,6,R superior LMS,0.8329295,FALSE
71,7,7,L superior LMS,1.2218516,FALSE
72,8,8,R inferior LMS,0.3445809,FALSE
73,9,9,L inferior LMS,0.7483035,FALSE
74,10,10,culmen,0.6659211,FALSE


In [7]:
# details on the environment
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS High Sierra 10.13.2

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libLAPACK.dylib

locale:
[1] en_CA.UTF-8/en_CA.UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] reshape2_1.4.3 digest_0.6.13  plyr_1.8.4    

loaded via a namespace (and not attached):
 [1] compiler_3.4.3  magrittr_1.5    IRdisplay_0.4.4 pbdZMQ_0.3-0   
 [5] tools_3.4.3     Rcpp_0.12.14    crayon_1.3.4    uuid_0.1-2     
 [9] stringi_1.1.6   IRkernel_0.8.11 jsonlite_1.5    stringr_1.2.0  
[13] repr_0.12.0     evaluate_0.10.1